# Generative Models
> "Algorithms that tries to learn mappings directly from space input $\\X$ to the labels $y$, or tries to map $p(y|x)$ directly, is called **discriminative learning algorithms**"
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [machine-learning, supervised, naive-bayes, gaussian-discriminant-analysis]

Algorithms that tries to learn mappings directly from space input $\\X$ to the labels $y$, or tries to map $p(y|x)$ directly, is called **discriminative learning algorithms**

Algorithms that tries to learn $p(x|y)$ then tries to infer $p(y|x)$ by bayes rules, is called **generative learning algorithms**.

Note: Bayes Rule:


$$
p(y|x) = \frac{p(x|y)p(y)}{p(x)}
$$

but then to calculate $p(y|x)$, we don't actually need to calculate denominator

$$\begin{eqnarray}
arg \space max_y p(y | x) &=& arg \space max_y \space \frac{p(x | y) p(y)}{p(x)}\\
&=&arg \space max_y p(x|y)p(y)
\end{eqnarray}$$


## Gaussian Discriminant Analysis

- assume that $p(x|y)$ distributed according to **multivariate normal distributions**
$$
p(x;\mu, \Sigma) = \frac{1}{(2\phi)^{d/2} |\Sigma|^{1/2}}exp \space \left(-\frac{1}{2}(x-\mu)^T \Sigma^{-1}(x-\mu) \right)
$$
- feature $x$ is continuous-valued random variables.

$$\begin{eqnarray}
y &\sim& Bernoulli(\phi)\\
x|y=0 &\sim& \mathcal{N}(\mu_0, \Sigma)\\
x|y=1 &\sim& \mathcal{N}(\mu_1, \Sigma)
\end{eqnarray}$$

### GDA and Logistic Regression

GDA Model:

$$
p(y=1 | x;\phi, \Sigma, \mu_0, \mu_1) = \frac{1}{1 + exp(-\theta^Tx)}
$$

where $\theta$ is appropriate function of $\phi,\Sigma, \mu_0, \mu_1$

while $p(x|y)$ is multivariate gaussian then $p(y|x)$ necessarily follows a logistic function.

But, if $p(y|x)$ being a logistic function does not imply $p(x|y)$ is multivariate gaussian.

Thus, 
>**GDA makes stronger modeling assumptions about the data than does logistic regression**.

if the assumption are correct (the data is gaussian), then GDA will find better fits to the data --> a better model, and GDA is asymptotically efficient.

or

if the training sets is very large, then there is no algorithm is strictly better than GDA.


**However,**
by making significantly weaker assumptions, logistic regression is more robust and less sensitive to incorrect modeling assumptions. For example if $x|y = 0 \sim \text{Poisson}(\lambda_0)$ and $x|y=1 \sim \text{Poisson}{lambda_1}$, then $p(y|x)$ will be logistic, but we won't get good result by fitting GDA.

Summary
> GDA makes stonger modeling assumptions, and is more data efficient **when the modeling assumptions are correct or at least approximately correct**. Logistic regression makes weaker assumptions, and **is significantly more robust to deviations from modeling assumptions**.

## Naive Bayes
GDA could be used if the feature vector $x$ is continuous, real valued vectors. What about discrete-valued? we will use classifying spam and ham example from lecture notes week 2 on CS229 by Andrew Ng.

to model $p(x|y)$, we will assume that $x_i$'s are conditionally independent given $y$. This is called **Naive Bayes assumption**, then the classifier is called **Naive Bayes classifier**.

$$
p(x|y) = \prod_{j=1}^{d}p(x_j|y)
$$
where

$j$ is the word in vocab index

$$
\mathcal{L}(\phi_y, \phi_{j|y=0}, \phi_{j|y=1}) = \prod_{i=1}^{n}p(x^{(i)}, y^{(i)})
$$

maximizing this w.r.t $\phi_y, \phi_{j|y=0}, \phi_{j|y=1}$ gives the maximum likelihood estimates:

$$\begin{eqnarray}
\phi_{j|y=1} &=& \frac{\sum_{i=1}^{n} 1 \{ x_j^{(i)} = 1 \wedge y^{(i)} = 1\}}{\sum_{j=1}^{n} 1 \{ y^{(i)} = 1\}} \\
\phi_{j|y=0} &=& \frac{\sum_{i=1}^{n} 1 \{ x_j^{(i)} = 1 \wedge y^{(i)} = 0\}}{\sum_{j=1}^{n} 1 \{ y^{(i)} = 0\}} \\
\phi_y &=& \frac{\sum_{i=1}^{n} 1\{ y^{(i)} = 1\}}{n}
\end{eqnarray}$$

read example: $\phi_{j|y=1}$ is the fraction of spam ($y=1$) emails in which word $j$ appear.

In [3]:
from collections import Counter
import numpy as np

In [138]:
def phi_x_given_y(x: np.array, y:np.array, condition_y:int)-> np.array:
    """
    Args
    ---
    x: array of documents, each has been splitted per token
    y: labels
    condition_y: conditioned label
    """
    def _combine(list_of_counters: list)-> dict:
        """
        producing map of word and total count in training data
        """
        result = dict()
        for word_counter in list_of_counters:            
            for word in word_counter:
                if word not in result:
                    result[word] = 0
                result[word] += word_counter[word]
        return result
    
    filtered_x = x[y==condition_y]
    combined = _combine(list(map(lambda s: Counter(s), filtered_x)))
    
    words, word_counts = (
        np.array(list(combined.keys())), 
        np.array(list(combined.values())))
    denominator = y[y==condition_y].shape[0]
    prob_xs_given_y = word_counts / np.float(np.sum(word_counts))
    return words, prob_xs_given_y

def phi_y(y: np.array, condition_y: int) -> float:
    return y[y==condition_y].shape[0] / np.float(y.shape[0])

def learn(x: np.array, y: np.array):
    all_possible_y = np.unique(y)
    p_x_given_ys = {
        yi: dict(zip(*phi_x_given_y(x, y, yi)))
        for yi in all_possible_y
    }
    p_ys = {
        yi: phi_y(y, yi)
        for yi in all_possible_y
    }

    return p_x_given_ys, p_ys

def h_x(p_x_given_ys: dict, p_ys: dict, x) -> int:
    words = np.unique(x)
    p_x_given_ys_res = np.zeros(len(p_x_given_ys.keys()))
    
    for yi in p_x_given_ys:
        p_x_given_y = 0
        for word in words:
            if word in p_x_given_ys[yi]:
                if p_x_given_y == 0:
                    p_x_given_y = 1
                p_x_given_y *= p_x_given_ys[yi][word]
        
        p_x_given_ys_res[yi] = p_x_given_y
        
    p_ys_arr = np.array(list(p_ys.values()))
    nominator = p_x_given_ys_res * p_ys_arr
    px = np.sum(p_x_given_ys_res * p_ys_arr) #denominator
    
    p_y_given_x = nominator/px
    max_i = np.argmax(p_y_given_x)
    return max_i, p_y_given_x[max_i]

In [139]:
x = np.array([
    'a a a b'.split(),
    'a b a a'.split(),
    'b b'.split(),
    'b b a'.split()
])
y = np.array([
    1,
    1,
    0,
    0
])

In [140]:
p_x_given_ys, p_ys = learn(x, y)

In [149]:
h_x(p_x_given_ys, p_ys, 'a '.split())

(1, 0.7894736842105263)